In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services. In this notebook, we use ChatGPT, Gemini on GenerativeAI and Anthropic Claude.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# How to input langrila's modules

In langrila, universal message class allows you to handle each client message in a same way.

# Import modules

In [3]:
from langrila.claude import Claude
from langrila.gemini import Gemini
from langrila.openai import ChatGPT

# Define tools

In this example, we can use these tools.

In [4]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    return f"Disco ball is {'spinning!' if power else 'stopped.'}"


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    return f"Starting music! {energetic=} {loud=}, {bpm=}"


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    return f"Lights are now set to {brightness}"

# Tool use

In langrila, you can define tool configs via ToolProperty, ToolParameter and ToolConfig object.

### ChatGPT

In [5]:
from langrila.openai import ToolConfig, ToolParameter, ToolProperty

chatgpt_tool_configs = [
    ToolConfig(
        name="power_disco_ball",
        description="Powers the spinning disco ball.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="power",
                    type="boolean",
                    description="Boolean to spin disco ball.",
                ),
            ],
            required=["power"],
        ),
    ),
    ToolConfig(
        name="start_music",
        description="Play some music matching the specified parameters.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="energetic",
                    type="boolean",
                    description="Whether the music is energetic or not.",
                ),
                ToolProperty(
                    name="loud", type="boolean", description="Whether the music is loud or not."
                ),
                ToolProperty(
                    name="bpm",
                    type="number",
                    description="The beats per minute of the music.",
                    enum=[60, 180],
                ),
            ],
            required=["energetic", "loud", "bpm"],
        ),
    ),
    ToolConfig(
        name="dim_lights",
        description="Dim the lights.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="brightness",
                    type="number",
                    description="The brightness of the lights, 0.0 is off, 1.0 is full.",
                ),
            ],
            required=["brightness"],
        ),
    ),
]

ToolConfig object is internally converted to client tool configs

In [6]:
# You can see the formatted tool configs by running the following
chatgpt_tool_configs[1].format()

{'type': 'function',
 'function': {'name': 'start_music',
  'description': 'Play some music matching the specified parameters.',
  'parameters': {'type': 'object',
   'properties': {'energetic': {'type': 'boolean',
     'description': 'Whether the music is energetic or not.'},
    'loud': {'type': 'boolean',
     'description': 'Whether the music is loud or not.'},
    'bpm': {'type': 'number',
     'description': 'The beats per minute of the music.',
     'enum': [60, 180]}},
   'required': ['energetic', 'loud', 'bpm']}}}

In [7]:
chatgpt = ChatGPT(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=chatgpt_tool_configs,
)

In [8]:
prompt = "Turn this place into a party mood!"

response = chatgpt.run(prompt)

# Response is CompletionResults object
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text='The party mood is on! 🎉\n\n- The disco ball is spinning, casting fun lights all around!\n- Energetic music with a lively beat of 180 BPM is blasting through the speakers!\n- The lights are dimmed to create just the right atmosphere!\n\nGet ready to dance and have a great time! 🕺💃')], name=None), usage=Usage(prompt_tokens=131, completion_tokens=69, total_tokens=200), prompt=[{'role': 'user', 'content': [{'type': 'text', 'text': 'Turn this place into a party mood!'}], 'name': None}, {'content': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_QUXnFJ4C9wnxr1l1VOSwzqrA', 'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'}, 'type': 'function'}, {'id': 'call_1EIfdLu7MhrLAAG3GSkgZzIu', 'function': {'arguments': '{"energetic": true, "loud": true, "bpm": 180}', 'name': 'start_music'}, 'type': 'function'}, {'id': 'call_Zugj9mVruw9WE03RKbrsKuNA', 'function': {'arguments

In [9]:
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is on! 🎉\n\n- The disco ball is spinning, casting fun lights all around!\n- Energetic music with a lively beat of 180 BPM is blasting through the speakers!\n- The lights are dimmed to create just the right atmosphere!\n\nGet ready to dance and have a great time! 🕺💃'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 131,
  'completion_tokens': 69},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text', 'text': 'Turn this place into a party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_QUXnFJ4C9wnxr1l1VOSwzqrA',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_1EIfdLu7MhrLAAG3GSkgZzIu',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": 180}',
      'name': 'start_music'},
     'type': 'function'

Asynchronous generation

In [10]:
response = await chatgpt.arun(prompt)
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text="The party mood is in full swing! 🎉 \n\n- **Music**: Energetic tracks are blasting at 180 BPM! 🎶\n- **Disco Ball**: The disco ball is spinning, adding that perfect sparkle! ✨\n- **Lights**: The lights are dimmed to a cozy half-brightness for that festive vibe! \n\nLet the celebrations begin! 🕺💃 What's next on the party agenda?")], name=None), usage=Usage(prompt_tokens=131, completion_tokens=91, total_tokens=222), prompt=[{'role': 'user', 'content': [{'type': 'text', 'text': 'Turn this place into a party mood!'}], 'name': None}, {'content': None, 'role': 'assistant', 'function_call': None, 'tool_calls': [{'id': 'call_DU81krTjApmku8RItZjdbh3s', 'function': {'arguments': '{"energetic": true, "loud": true, "bpm": 180}', 'name': 'start_music'}, 'type': 'function'}, {'id': 'call_3envFHq50HayaVmV515O0WXB', 'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'}, 'type': 'function'}, {'id': 'call_UbQ

multiple generation

In [10]:
response = chatgpt.run(prompt, n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': 'The party mood is in full swing! 🎉 The disco ball is spinning, the energetic music is blasting at 180 BPM, and the lights are dimmed just right to create that perfect vibe. Let’s get this celebration started! 🍾🕺💃'},
  {'text': 'The party vibe is on! The disco ball is spinning, energetic music is blasting at 180 BPM, and the lights are dimmed to set the perfect mood. Let’s get this celebration started! 🕺🎉✨'}],
 'name': None}

You can get the results from specific tool.

In [11]:
prompt = "Turn this place into a party mood!"

response = chatgpt.run(prompt, tool_choice="start_music")

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The music is pumping at 180 BPM! Let’s set the scene for an unforgettable party:\n\n🎉 **Lighting:** Dim the overhead lights and add colorful LED lights to create a vibrant atmosphere. Maybe even some disco balls for that retro vibe!\n\n🎈 **Decorations:** Blow up balloons and hang streamers. You could set up a photo booth with fun props for some memorable snapshots!\n\n🍹 **Drinks:** Set up a drinks station with colorful cocktails and mocktails, along with some fun drinkware. Don’t forget a punch bowl for that extra party flair!\n\n🍕 **Snacks:** Arrange a buffet of easy-to-eat finger foods like sliders, nachos, and veggie platters. Party food that lets people mingle!\n\n🎶 **Dance Floor:** Clear a space for dancing! Encourage everyone to show off their best moves as the bass drops.\n\n📅 **Games:** Set up some fun party games like a dance-off or a karaoke machine to keep the energy high!\n\nWith all that set, let the party begin! 🎊💃

If you specify tool_only option as True, you get the FunctionCallingResults object instead of CompletionResults object.

In [12]:
response = chatgpt.run(prompt, tool_only=True)
response

FunctionCallingResults(usage=Usage(prompt_tokens=163, completion_tokens=75, total_tokens=238), results=[Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_BRVYgc6Ql42Mz6lcLfdajWCY', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball'), Message(role='function', content=[ToolContent(output='Starting music! energetic=True loud=True, bpm=180', call_id='call_SmBzqbnmB6JlVdt3D5hEWfH8', args='{"energetic": true, "loud": true, "bpm": 180}', funcname='start_music')], name='start_music'), Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_3iCL7TT2f3lgEVa8K1cOElHr', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights')], calls=Message(role='function_call', content=[ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_BRVYgc6Ql42Mz6lcLfdajWCY'), ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": 180}', call_id='call_SmBzqbn

In [13]:
response.model_dump()

{'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 163,
  'completion_tokens': 75},
 'results': [{'role': 'function',
   'content': [{'output': 'Disco ball is spinning!',
     'call_id': 'call_BRVYgc6Ql42Mz6lcLfdajWCY',
     'args': '{"power": true}',
     'funcname': 'power_disco_ball'}],
   'name': 'power_disco_ball'},
  {'role': 'function',
   'content': [{'output': 'Starting music! energetic=True loud=True, bpm=180',
     'call_id': 'call_SmBzqbnmB6JlVdt3D5hEWfH8',
     'args': '{"energetic": true, "loud": true, "bpm": 180}',
     'funcname': 'start_music'}],
   'name': 'start_music'},
  {'role': 'function',
   'content': [{'output': 'Lights are now set to 0.5',
     'call_id': 'call_3iCL7TT2f3lgEVa8K1cOElHr',
     'args': '{"brightness": 0.5}',
     'funcname': 'dim_lights'}],
   'name': 'dim_lights'}],
 'calls': {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_BRVYgc6Ql42Mz6lcLfdajWCY'},
  

In [14]:
# Tool result messages

response.results

[Message(role='function', content=[ToolContent(output='Disco ball is spinning!', call_id='call_BRVYgc6Ql42Mz6lcLfdajWCY', args='{"power": true}', funcname='power_disco_ball')], name='power_disco_ball'),
 Message(role='function', content=[ToolContent(output='Starting music! energetic=True loud=True, bpm=180', call_id='call_SmBzqbnmB6JlVdt3D5hEWfH8', args='{"energetic": true, "loud": true, "bpm": 180}', funcname='start_music')], name='start_music'),
 Message(role='function', content=[ToolContent(output='Lights are now set to 0.5', call_id='call_3iCL7TT2f3lgEVa8K1cOElHr', args='{"brightness": 0.5}', funcname='dim_lights')], name='dim_lights')]

In [15]:
# Tool call messages

response.calls

Message(role='function_call', content=[ToolCall(name='power_disco_ball', args='{"power": true}', call_id='call_BRVYgc6Ql42Mz6lcLfdajWCY'), ToolCall(name='start_music', args='{"energetic": true, "loud": true, "bpm": 180}', call_id='call_SmBzqbnmB6JlVdt3D5hEWfH8'), ToolCall(name='dim_lights', args='{"brightness": 0.5}', call_id='call_3iCL7TT2f3lgEVa8K1cOElHr')], name=None)

### Gemini on GenerativeAI

In [12]:
from langrila.gemini.genai import ToolConfig, ToolParameter, ToolProperty

# NOTE : following tool_configs is almost same as that of OpenAI
gemini_tool_configs = [
    ToolConfig(
        name="power_disco_ball",
        description="Powers the spinning disco ball.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="power",
                    type="boolean",
                    description="Boolean to spin disco ball.",
                ),
            ],
            required=["power"],
        ),
    ),
    ToolConfig(
        name="start_music",
        description="Play some music matching the specified parameters.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="energetic",
                    type="boolean",
                    description="Whether the music is energetic or not.",
                ),
                ToolProperty(
                    name="loud", type="boolean", description="Whether the music is loud or not."
                ),
                ToolProperty(
                    name="bpm",
                    type="string",
                    description="The beats per minute of the music.",
                    enum=["60", "180", "240"],  # gemini does not support numerical enum
                ),
            ],
            required=["energetic", "loud", "bpm"],
        ),
    ),
    ToolConfig(
        name="dim_lights",
        description="Dim the lights.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="brightness",
                    type="number",
                    description="The brightness of the lights, 0.0 is off, 1.0 is full.",
                ),
            ],
            required=["brightness"],
        ),
    ),
]

In [13]:
# You can see the formatted tool config as follows
gemini_tool_configs[1].format()

name: "start_music"
description: "Play some music matching the specified parameters."
parameters {
  type_: OBJECT
  properties {
    key: "loud"
    value {
      type_: BOOLEAN
      description: "Whether the music is loud or not."
    }
  }
  properties {
    key: "energetic"
    value {
      type_: BOOLEAN
      description: "Whether the music is energetic or not."
    }
  }
  properties {
    key: "bpm"
    value {
      type_: STRING
      description: "The beats per minute of the music."
      enum: "60"
      enum: "180"
      enum: "240"
    }
  }
  required: "energetic"
  required: "loud"
  required: "bpm"
}

In [14]:
gemini = Gemini(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=gemini_tool_configs,
)

In [15]:
prompt = "Turn this place into a party!"
response = gemini.run(prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "The disco ball is ALIVE!  Music is pumping and the lights are low. Let's get this party started! 🎉🥳 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 143,
  'completion_tokens': 26},
 'prompt': [parts {
    text: "Turn this place into a party!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "180"
          }
        }
      }
    }
  }
  parts {
    function

Asynchronous generation

In [16]:
response = await gemini.arun(prompt)

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'Alright, the disco ball is spinning, the music is pumping, and the lights are dimmed! Time to get this party started! 🎉 \U0001faa9 🎶 \n'}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 143,
  'completion_tokens': 31},
 'prompt': [parts {
    text: "Turn this place into a party!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "180"
          }
        }
      }

Gemini allows you to call specific tool as well as ChatGPT.

In [21]:
prompt = "Turn this place into a party!"
response = gemini.run(prompt, tool_choice="start_music")

# show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, everybody get on the floor!  Let's get this party STARTED! 🎉🥳 🎶 \n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 66,
  'completion_tokens': 21},
 'prompt': [parts {
    text: "Turn this place into a party!"
  }
  role: "user",
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            string_value: "240"
          }
        }
      }
    }
  }
  role: "model",
  parts {
    function_response {
      name: "start_music"
      response {
        fields {
          key: "content"
          value {
            string_value: "Starting music! energetic=True loud=True, bpm=\'240\'"
          }
   

### Gemini on VertexAI

In [17]:
from langrila.gemini.vertexai import ToolConfig, ToolParameter, ToolProperty

# NOTE : following tool_configs is almost same as that of OpenAI
gemini_tool_configs = [
    ToolConfig(
        name="power_disco_ball",
        description="Powers the spinning disco ball.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="power",
                    type="boolean",
                    description="Boolean to spin disco ball.",
                ),
            ],
            required=["power"],
        ),
    ),
    ToolConfig(
        name="start_music",
        description="Play some music matching the specified parameters.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="energetic",
                    type="boolean",
                    description="Whether the music is energetic or not.",
                ),
                ToolProperty(
                    name="loud", type="boolean", description="Whether the music is loud or not."
                ),
                ToolProperty(
                    name="bpm",
                    type="string",
                    description="The beats per minute of the music.",
                    enum=["60", "180", "240"],  # gemini does not support numerical enum
                ),
            ],
            required=["energetic", "loud", "bpm"],
        ),
    ),
    ToolConfig(
        name="dim_lights",
        description="Dim the lights.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="brightness",
                    type="number",
                    description="The brightness of the lights, 0.0 is off, 1.0 is full.",
                ),
            ],
            required=["brightness"],
        ),
    ),
]

In [20]:
gemini_vertexai = Gemini(
    model_name="gemini-1.5-flash",
    api_type="vertexai",
    project_id_env_name="PROJECT_ID",
    location_env_name="LOCATION",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=gemini_tool_configs,
)

In [21]:
response = gemini_vertexai.run("Turn this place into a party!")

In [23]:
response

CompletionResults(message=Message(role='assistant', content=[TextContent(text="Okay! I'm turning this place into a party! 🎉\n\nI've already started the music - it's high energy, loud, and ready to get everyone moving! 🎶\n\nThe disco ball is spinning, casting shimmering lights everywhere, and the lights have been dimmed to create the perfect party atmosphere! \U0001faa9\n\nWhat else can I do to make this the best party ever? Tell me:\n\n* **What kind of music do you want?** (e.g., pop, hip hop, electronic)\n* **Do you want any snacks or drinks?**\n* **What kind of activities or games should we play?**\n* **Do you want any decorations?** \n\nLet's get this party started! 🥳 \n")], name=None), usage=Usage(prompt_tokens=71, completion_tokens=156, total_tokens=227), prompt=[role: "user"
parts {
  text: "Turn this place into a party!"
}
, role: "model"
parts {
  function_call {
    name: "power_disco_ball"
    args {
      fields {
        key: "power"
        value {
          bool_value: tr

multiple generation

In [24]:
response = gemini_vertexai.run("Turn this place into a party!", n_results=2)
response.message.model_dump()

{'role': 'assistant',
 'content': [{'text': "The disco ball is spinning, the music is pumping, and the lights are dimmed for a perfect party atmosphere!  \n\nLet's get this party started! \n\nWhat else can I do to make this the best party ever?  Do you want me to:\n\n* **Suggest some party games?**\n* **Find some fun party recipes?**\n* **Create a playlist?**\n* **Decorate the space?** \n* **Anything else!**\n\nJust let me know! 🎉 \n"},
  {'text': "The disco ball is spinning, the music is pumping, and the lights are dimmed for a perfect party atmosphere! Let's get this party started! 🎉 \n"}],
 'name': None}

### Anthropic Claude

In [25]:
from langrila.claude import ToolConfig, ToolParameter, ToolProperty

# NOTE : following tool_configs is the same as that of OpenAI
claude_tool_configs = [
    ToolConfig(
        name="power_disco_ball",
        description="Powers the spinning disco ball.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="power",
                    type="boolean",
                    description="Boolean to spin disco ball.",
                ),
            ],
            required=["power"],
        ),
    ),
    ToolConfig(
        name="start_music",
        description="Play some music matching the specified parameters.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="energetic",
                    type="boolean",
                    description="Whether the music is energetic or not.",
                ),
                ToolProperty(
                    name="loud", type="boolean", description="Whether the music is loud or not."
                ),
                ToolProperty(
                    name="bpm",
                    type="number",
                    description="The beats per minute of the music.",
                    enum=[60, 180],
                ),
            ],
            required=["energetic", "loud", "bpm"],
        ),
    ),
    ToolConfig(
        name="dim_lights",
        description="Dim the lights.",
        parameters=ToolParameter(
            properties=[
                ToolProperty(
                    name="brightness",
                    type="number",
                    description="The brightness of the lights, 0.0 is off, 1.0 is full.",
                ),
            ],
            required=["brightness"],
        ),
    ),
]

In [26]:
# You can see the formatted tool config as follows
claude_tool_configs[1].format()

{'name': 'start_music',
 'description': 'Play some music matching the specified parameters.',
 'input_schema': {'type': 'object',
  'properties': {'energetic': {'type': 'boolean',
    'description': 'Whether the music is energetic or not.'},
   'loud': {'type': 'boolean',
    'description': 'Whether the music is loud or not.'},
   'bpm': {'type': 'number',
    'description': 'The beats per minute of the music.',
    'enum': [60, 180]}},
  'required': ['energetic', 'loud', 'bpm']}}

In [27]:
claude = Claude(
    model_name="claude-3-5-sonnet-20240620",
    api_key_env_name="ANTHROPIC_API_KEY",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=claude_tool_configs,
)

In [28]:
prompt = "Turn this place into a party!"

response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! The disco ball is now spinning, adding a fun, party atmosphere to the room.\n\n2. Next, let's start some energetic music. For a party, we usually want upbeat, energetic music with a faster tempo. Let's go with the highest available BPM and make it loud:"}],
  'name': None},
 'usage': {'model_name': 'claude-3-5-sonnet-20240620',
  'prompt_tokens': 721,
  'completion_tokens': 156},
 'prompt': [{'role': 'user',
   'content': [{'text': 'Turn this place into a party!', 'type': 'text'}]},
  {'role': 'assistant',
   'content': [{'text': "Certainly! I'd be happy to help turn this place into a party using the available tools. To create a party atmosphere, we'll want to use the disco ball, play some upbeat music, and adjust the lighting. Let's do this step by step:\n\n1. First, let's power on the disco ball:",
     'type': 'text'},
    {'id': 'toolu_01Dft7Z88T7wcwzZG21JfxKx',
     'type': 'tool_use',
     'input': {'power': True},


Asynchronous generation

In [29]:
response = await claude.arun(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up the perfect party atmosphere for you. Here's what I've done:\n\n1. Powered on the disco ball: The spinning disco ball will create a fun, dynamic lighting effect that's perfect for a party.\n\n2. Started some energetic music: I've chosen high-energy, loud music with a fast tempo (180 BPM) to get everyone in the party mood and encourage dancing.\n\n3. Adjusted the lighting: I've dimmed the lights to 60% brightness (0.6). This creates a nice balance – it's dark enough to create a party atmosphere but still bright enough for people to see and move around safely.\n\nThese elements combined should create an exciting party environment. The spinning disco ball will reflect light around the room, the energetic music will get people moving, and the dimmed lights will set the right mood.\n\nIs there anything else you'd like me to adjust to make the party even better? I can change the music style, adjust the lighting, or 

### Claude on Amazon Bedrock

In [31]:
claude_bedrock = Claude(
    model_name="anthropic.claude-3-sonnet-20240229-v1:0",
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=claude_tool_configs,
)

In [32]:
prompt = "Turn this place into a party!"

response = claude.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Great! I've set up the perfect party atmosphere for you. Here's what I've done:\n\n1. Disco Ball: I've turned on the spinning disco ball to create a fun, party-like atmosphere with moving lights.\n\n2. Music: I've started playing energetic and loud music with a fast tempo of 180 beats per minute. This should get everyone in the mood to dance and have a good time.\n\n3. Lighting: I've dimmed the lights to 60% brightness (0.6). This creates a nice balance between having enough light to see and dance, while also setting a party mood.\n\nThese elements combined should create an exciting party environment. The spinning disco ball will reflect light around the room, the energetic music will encourage dancing, and the dimmed lights will set the right ambiance.\n\nIs there anything else you'd like me to adjust to make the party even better? I can change the music, adjust the lighting, or turn off the disco ball if needed. Just let me kn

# Multi-turn conversation with tools

In [27]:
from langrila import InMemoryConversationMemory, JSONConversationMemory

chatgpt = ChatGPT(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=chatgpt_tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    # conversation_memory=JSONConversationMemory("./test.json"), # for serialization
)

In [28]:
prompt = "Turn this place into a party mood!"

response = chatgpt.run(prompt)

# Show result
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is on! 🎉 The disco ball is spinning, energetic music is pumping at 180 BPM, and the lights are dimmed to create the perfect atmosphere. Let the fun begin!'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 131,
  'completion_tokens': 40},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text', 'text': 'Turn this place into a party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_gpooMTVmudPH3Mrf6aeyYRwn',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_MQeblbc6JcGOADAlbZBkaGBm',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": 180}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_uUx9qDYqk9B2ytpl2pF7dA3A',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'd

Next turn

In [29]:
prompt = "Can you make bpm up more?"

response = chatgpt.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The BPM has just been cranked up to 240! ⚡ Get ready to dance and feel the energy surge through the room! Let’s keep the party going! 🎶💃🕺'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 231,
  'completion_tokens': 41},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text', 'text': 'Turn this place into a party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_gpooMTVmudPH3Mrf6aeyYRwn',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_MQeblbc6JcGOADAlbZBkaGBm',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": 180}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_uUx9qDYqk9B2ytpl2pF7dA3A',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
     'type': 'funct

## Multi-turn conversation using tools with multiple model

In [30]:
from langrila import InMemoryConversationMemory

shared_memory = InMemoryConversationMemory()

chatgpt = ChatGPT(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=chatgpt_tool_configs,
    conversation_memory=shared_memory,
)

gemini = Gemini(
    api_key_env_name="GEMINI_API_KEY",
    model_name="gemini-1.5-pro",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=gemini_tool_configs,
    conversation_memory=shared_memory,
)

In [31]:
prompt = "Turn this place into a party mood!"

response = chatgpt.run(prompt)

In [32]:
response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': 'The party mood is set! The disco ball is spinning, energetic music is blasting at 180 BPM, and the lights are dimmed just right. Let the fun begin! 🎉💃🕺'}],
  'name': None},
 'usage': {'model_name': 'gpt-4o-mini-2024-07-18',
  'prompt_tokens': 131,
  'completion_tokens': 42},
 'prompt': [{'role': 'user',
   'content': [{'type': 'text', 'text': 'Turn this place into a party mood!'}],
   'name': None},
  {'content': None,
   'role': 'assistant',
   'function_call': None,
   'tool_calls': [{'id': 'call_ex3oj8l4LkroCeQR0hxF98xO',
     'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
     'type': 'function'},
    {'id': 'call_uF1CH7D3aLK6T5obIrBPczzs',
     'function': {'arguments': '{"energetic": true, "loud": true, "bpm": 180}',
      'name': 'start_music'},
     'type': 'function'},
    {'id': 'call_0wlX1PTPkh0ACIZn8PeUszVR',
     'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
     

Gemini can continue conversation without any additional process.

In [33]:
prompt = "Can you make bpm up more?"

response = gemini.run(prompt=prompt)

response.model_dump()

{'message': {'role': 'assistant',
  'content': [{'text': "Alright, things are about to get even more intense! I've cranked the BPM up to a thumping 240.  Get ready to move! 🌪️🎶\n"}],
  'name': None},
 'usage': {'model_name': 'gemini-1.5-pro',
  'prompt_tokens': 250,
  'completion_tokens': 38},
 'prompt': [parts {
    text: "Turn this place into a party mood!"
  }
  role: "user",
  parts {
    function_call {
      name: "power_disco_ball"
      args {
        fields {
          key: "power"
          value {
            bool_value: true
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "start_music"
      args {
        fields {
          key: "loud"
          value {
            bool_value: true
          }
        }
        fields {
          key: "energetic"
          value {
            bool_value: true
          }
        }
        fields {
          key: "bpm"
          value {
            number_value: 180
          }
        }
      }
    }
  }
 

Gemini also works as well.

NOTE : Claude may not work in multi-turn conversation using tools. Bug fix is in progress.

# Multi-turn conversation with tools and context truncation

Although context truncation doesn't work accurately if we use tools, but truncation is useful for token management to some extent.

In [34]:
from langrila import InMemoryConversationMemory

chatgpt = ChatGPT(
    api_key_env_name="API_KEY",
    model_name="gpt-4o-mini-2024-07-18",
    tools=[power_disco_ball, start_music, dim_lights],
    tool_configs=chatgpt_tool_configs,
    conversation_memory=InMemoryConversationMemory(),
    context_length=50,
)

In [35]:
prompt = "Turn this place into a party mood!"

response = chatgpt.run(prompt, tool_choice="auto")

In [36]:
prompt = "Can you make bpm up more?"

response = chatgpt.run(prompt=prompt, tool_choice="auto")

Input message is truncated because total length of messages exceeds context length.
Input message is truncated because total length of messages exceeds context length.


User message and assistant message are target of truncation while tool call message and tool result message is not.

In [37]:
print(response.message.content[0].text)

The BPM is now cranked up to 240! Let’s keep the energy high and the dance floor rocking! 🎶🔥💃


In [38]:
response.usage

Usage(prompt_tokens=208, completion_tokens=28, total_tokens=236)

In [39]:
response.prompt

[{'content': None,
  'role': 'assistant',
  'function_call': None,
  'tool_calls': [{'id': 'call_TcDwO1BDhSOVdkFBoEyfwnDT',
    'function': {'arguments': '{"power": true}', 'name': 'power_disco_ball'},
    'type': 'function'},
   {'id': 'call_jTQDuenDH5XUsblX2VASWWQK',
    'function': {'arguments': '{"energetic": true, "loud": true, "bpm": 180}',
     'name': 'start_music'},
    'type': 'function'},
   {'id': 'call_ZU9T9HbEev1gfcduKAJKRIA3',
    'function': {'arguments': '{"brightness": 0.5}', 'name': 'dim_lights'},
    'type': 'function'}]},
 {'role': 'tool',
  'tool_call_id': 'call_TcDwO1BDhSOVdkFBoEyfwnDT',
  'name': 'power_disco_ball',
  'content': 'Disco ball is spinning!'},
 {'role': 'tool',
  'tool_call_id': 'call_jTQDuenDH5XUsblX2VASWWQK',
  'name': 'start_music',
  'content': 'Starting music! energetic=True loud=True, bpm=180'},
 {'role': 'tool',
  'tool_call_id': 'call_ZU9T9HbEev1gfcduKAJKRIA3',
  'name': 'dim_lights',
  'content': 'Lights are now set to 0.5'},
 {'role': 'ass

In [40]:
# whole history
chatgpt.conversation_memory.load()

[{'role': 'user',
  'content': [{'text': 'Turn this place into a party mood!'}],
  'name': None},
 {'role': 'function_call',
  'content': [{'name': 'power_disco_ball',
    'args': '{"power": true}',
    'call_id': 'call_TcDwO1BDhSOVdkFBoEyfwnDT'},
   {'name': 'start_music',
    'args': '{"energetic": true, "loud": true, "bpm": 180}',
    'call_id': 'call_jTQDuenDH5XUsblX2VASWWQK'},
   {'name': 'dim_lights',
    'args': '{"brightness": 0.5}',
    'call_id': 'call_ZU9T9HbEev1gfcduKAJKRIA3'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': 'Disco ball is spinning!',
    'call_id': 'call_TcDwO1BDhSOVdkFBoEyfwnDT',
    'args': '{"power": true}',
    'funcname': 'power_disco_ball'}],
  'name': None},
 {'role': 'function',
  'content': [{'output': 'Starting music! energetic=True loud=True, bpm=180',
    'call_id': 'call_jTQDuenDH5XUsblX2VASWWQK',
    'args': '{"energetic": true, "loud": true, "bpm": 180}',
    'funcname': 'start_music'}],
  'name': None},
 {'role': 'function'